# Condensación de registros

La idea de esta notebook es condensar los registros de manera tal que quede un registro por usuario.

 - [screen_resolution](#screen_resolution)

In [1]:
%run limpieza.ipynb
df = get_clean_df()

Creo el nuevo dataframe que va a tener una fila por usuario:

In [10]:
c_df = pd.DataFrame(index=df['person'].unique())
c_df.index.name = 'person'

In [11]:
c_df.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


<h4 id="screen_resolution">screen_resolution</h4>

Para este atributos se utilizó el promedio entre los distintos valores, sin tener en cuenta los nulos.

In [130]:
c_df['screen_resolution_height'] = df.groupby('person')['screen_resolution_height'].mean()
c_df['screen_resolution_width'] = df.groupby('person')['screen_resolution_width'].mean()